In [80]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas as pd, xgboost, numpy as np, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

In [81]:
news_scraping_results = pd.read_csv("./news_scraping_results_and_label.csv")

In [82]:
news_scraping_results = news_scraping_results.dropna(subset=['sentiment'])

In [83]:
tickers_list = pd.read_csv("./tickers_list.csv")

In [84]:
suspended_ticker_news = news_scraping_results[news_scraping_results['SYMBOL'].isin(tickers_list['Symbol'].values)]

In [143]:
active_ticker_news = news_scraping_results[~news_scraping_results['SYMBOL'].isin(tickers_list['Symbol'].values)]

In [145]:
active_ticker_news.index.values

2018

In [146]:
len(X)

711

In [86]:
balanced_news = news_scraping_results[news_scraping_results['sentiment'] == 'positive'].reset_index()
balanced_news = balanced_news.append(news_scraping_results[news_scraping_results['sentiment'] == 'negative'].reset_index()[:237], ignore_index=True)
balanced_news = balanced_news.append(news_scraping_results[news_scraping_results['sentiment'] == 'neutral'].reset_index()[:237], ignore_index=True)

In [88]:
X, y = balanced_news[['CONTENT']].values.flatten().astype(str), balanced_news[['sentiment']].values.flatten().astype(str)

In [133]:
#dataset without suspended stock 


'welcome to morningstar co uk -pron- have be redirect here from hemscott com as -pron- be merge -pron- website to provide -pron- with a one stop shop for all -pron- investment research need to search for a security type the name or ticker in the search box at the top of the page and select from the dropdown result register hemscott user can log in to morningstar use the same login detail similarly if -pron- be a hemscott premium user -pron- now have a morningstar premium account which -pron- can access use the same login detail'

In [89]:
# split the dataset into training and validation datasets 
train_X, test_X, train_y, test_y = model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

In [90]:
# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
test_y = encoder.fit_transform(test_y)

In [91]:
encode_y = encoder.fit_transform(y)

In [92]:
#encode_y.classes_

In [93]:
#pd.DataFrame(encode_y, columns=['sentiment']).to_csv('balanced_encoded_labels.csv')

In [94]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(X)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='\\w{1,}', tokenizer=None,
                vocabulary=None)

In [95]:
#count_vect.vocabulary_

In [96]:
vector = count_vect.transform(X)
# summarize encoded vector
#print(vector.shape)


In [70]:
#pd.DataFrame(vector.toarray()).to_csv('balanced_count_vectorized_CONTENT.csv')

In [97]:
# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_X)
xtest_count =  count_vect.transform(test_X)

In [98]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=None)
tfidf_vect.fit(X)
xtrain_tfidf =  tfidf_vect.transform(train_X)
xtest_tfidf =  tfidf_vect.transform(test_X)

In [99]:
#tfidf_vect.vocabulary_

In [100]:
#tfidf_vect.idf_

In [101]:
# encode document
vector = tfidf_vect.transform(X)
# summarize encoded vector
#print(vector.shape)
#print(vector.toarray())

In [102]:
#pd.DataFrame(vector.toarray()).to_csv('balanced_tfidf_vectorized_CONTENT.csv')

In [103]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    accuracy = metrics.accuracy_score(test_y, predictions)
    precision = metrics.precision_score(test_y, predictions, average="macro")
    recall = metrics.recall_score(test_y, predictions, average="macro")
    f1 = metrics.f1_score(test_y, predictions, average="macro")
    
    return accuracy,precision,recall,f1

In [104]:
#evaluation_df = pd.DataFrame(columns=['classifier', 'feature vector', 'accuracy', 'precision', 'recall', 'f1'])

In [105]:
# Naive Bayes on Count Vectors
accuracy,precision,recall,f1 = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xtest_count)
#metrics = pd.DataFrame([['Naive Bayes', 'Count', accuracy, precision, recall, f1]], columns=['classifier', 'feature vector', 'accuracy', 'precision', 'recall', 'f1'])
#evaluation_df = evaluation_df.append(metrics)
#print("Naive Bayes, Count Vectors: \n accuracy: %s\n precision: %s\n recall: %s\n f1:%s"%(accuracy,precision,recall,f1))

Naive Bayes, Count Vectors: 
 accuracy: 0.5524475524475524
 precision: 0.5510528923572401
 recall: 0.555023923444976
 f1:0.5523099816045086


In [108]:
#metrics = pd.DataFrame([['Naive Bayes', 'Count', accuracy, precision, recall, f1]], columns=['classifier', 'feature vector', 'accuracy', 'precision', 'recall', 'f1'])
#evaluation_df = evaluation_df.append(metrics)
nb_count = np.array(['Naive Bayes', 'Count', accuracy, precision, recall, f1])

In [106]:
# Naive Bayes on Word Level TF IDF Vectors
accuracy,precision,recall,f1 = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xtest_tfidf)
#metrics = pd.DataFrame([['Naive Bayes', 'TF-IDF', accuracy, precision, recall, f1]], columns=['classifier', 'feature vector', 'accuracy', 'precision', 'recall', 'f1'])
#evaluation_df = evaluation_df.append(metrics)
#print("Naive Bayes, TF-IDF: \n accuracy: %s\n precision: %s\n recall: %s\n f1:%s"%(accuracy,precision,recall,f1))

Naive Bayes, TF-IDF: 
 accuracy: 0.5454545454545454
 precision: 0.593192195931922
 recall: 0.574884180147338
 f1:0.5318657127132637


In [109]:
#metrics = pd.DataFrame([['Naive Bayes', 'TF-IDF', accuracy, precision, recall, f1]], columns=['classifier', 'feature vector', 'accuracy', 'precision', 'recall', 'f1'])
#evaluation_df = evaluation_df.append(metrics)
nb_tfidf = np.array(['Naive Bayes', 'TF-IDF', accuracy, precision, recall, f1])

In [107]:
# Linear Classifier on Count Vectors
accuracy,precision,recall,f1 = train_model(linear_model.LogisticRegression(), xtrain_count, train_y, xtest_count)
#metrics = pd.DataFrame([['Logisitic Regression', 'Count', accuracy, precision, recall, f1]], columns=['classifier', 'feature vector', 'accuracy', 'precision', 'recall', 'f1'])
#evaluation_df = evaluation_df.append(metrics)
#print("linear Classifier, Count Vectors: \n accuracy: %s\n precision: %s\n recall: %s\n f1:%s"%(accuracy,precision,recall,f1))

/home/albert/anaconda3/envs/cs/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/albert/anaconda3/envs/cs/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


linear Classifier, Count Vectors: 
 accuracy: 0.4965034965034965
 precision: 0.4946524064171123
 recall: 0.4998860788334472
 f1:0.49595959595959593


In [110]:
lr_count = np.array(['Logistic Regression', 'Count', accuracy, precision, recall, f1])

In [518]:
# Linear Classifier on Word Level TF IDF Vectors
accuracy,precision,recall,f1 = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xtest_tfidf)
#metrics = pd.DataFrame([['Logisitic Regression', 'TF-IDF', accuracy, precision, recall, f1]], columns=['classifier', 'feature vector', 'accuracy', 'precision', 'recall', 'f1'])
#evaluation_df = evaluation_df.append(metrics)
#print("linear Classifier, TF-IDF: \n accuracy: %s\n precision: %s\n recall: %s\n f1:%s"%(accuracy,precision,recall,f1))

linear Classifier, TF-IDF: 
 accuracy: 0.5594405594405595
 precision: 0.5727339181286549
 recall: 0.5733272575377839
 f1:0.5618705730222144


In [111]:
lr_tfidf = np.array(['Logistic Regression', 'TF-IDF', accuracy, precision, recall, f1])

In [519]:
# Random Forest on Count Vectors
accuracy,precision,recall,f1 = train_model(ensemble.RandomForestClassifier(), xtrain_count, train_y, xtest_count)
#metrics = pd.DataFrame([['Random Forest', 'Count', accuracy, precision, recall, f1]], columns=['classifier', 'feature vector', 'accuracy', 'precision', 'recall', 'f1'])
#evaluation_df = evaluation_df.append(metrics)
#print("Random Forest, Count Vectors: \n accuracy: %s\n precision: %s\n recall: %s\n f1:%s"%(accuracy,precision,recall,f1))

Random Forest, Count Vectors: 
 accuracy: 0.5524475524475524
 precision: 0.5584074449322677
 recall: 0.5657514999620262
 f1:0.5556466599805284
Random Forest, WordLevel TF-IDF: 
 accuracy: 0.4755244755244755
 precision: 0.49490183491146417
 recall: 0.49443684970000756
 f1:0.47485808692705245


/home/albert/anaconda3/envs/cs/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/albert/anaconda3/envs/cs/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [112]:
rf_count = np.array(['Random Forest', 'Count', accuracy, precision, recall, f1])

In [ ]:
# RF on Word Level TF IDF Vectors
accuracy,precision,recall,f1 = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf, train_y, xtest_tfidf)
#metrics = pd.DataFrame([['Random Forest', 'TF-IDF', accuracy, precision, recall, f1]], columns=['classifier', 'feature vector', 'accuracy', 'precision', 'recall', 'f1'])
#evaluation_df = evaluation_df.append(metrics)
#print("Random Forest, WordLevel TF-IDF: \n accuracy: %s\n precision: %s\n recall: %s\n f1:%s"%(accuracy,precision,recall,f1))


In [113]:
rf_tfidf = np.array(['Random Forest', 'TF-IDF', accuracy, precision, recall, f1])

In [520]:
# Extereme Gradient Boosting on Count Vectors
accuracy,precision,recall,f1 = train_model(xgboost.XGBClassifier(), xtrain_count.tocsc(), train_y, xtest_count.tocsc())
#metrics = pd.DataFrame([['Extreme Gradient Boosting', 'Count', accuracy, precision, recall, f1]], columns=['classifier', 'feature vector', 'accuracy', 'precision', 'recall', 'f1'])
#evaluation_df = evaluation_df.append(metrics)
#print("Xgb, Count Vectors: \n accuracy: %s\n precision: %s\n recall: %s\n f1:%s"%(accuracy,precision,recall,f1))

# Extereme Gradient Boosting on Word Level TF IDF Vectors
accuracy,precision,recall,f1 = train_model(xgboost.XGBClassifier(), xtrain_tfidf.tocsc(), train_y, xtest_tfidf.tocsc())
#metrics = pd.DataFrame([['Extreme Gradient Boosting', 'TF-IDF', accuracy, precision, recall, f1]], columns=['classifier', 'feature vector', 'accuracy', 'precision', 'recall', 'f1'])
#evaluation_df = evaluation_df.append(metrics)
#print("Xgb, WordLevel TF-IDF: \n accuracy: %s\n precision: %s\n recall: %s\n f1:%s"%(accuracy,precision,recall,f1))

Xgb, Count Vectors: 
 accuracy: 0.5664335664335665
 precision: 0.573721340388007
 recall: 0.5875294296346928
 f1:0.5654577048210588
Xgb, WordLevel TF-IDF: 
 accuracy: 0.5384615384615384
 precision: 0.5411802232854864
 recall: 0.5575112022480443
 f1:0.5393315304070453


In [114]:
xgb_count = np.array(['Extreme Gradient Boosting', 'Count', accuracy, precision, recall, f1])

In [ ]:
# Extereme Gradient Boosting on Word Level TF IDF Vectors
accuracy,precision,recall,f1 = train_model(xgboost.XGBClassifier(), xtrain_tfidf.tocsc(), train_y, xtest_tfidf.tocsc())
#metrics = pd.DataFrame([['Extreme Gradient Boosting', 'TF-IDF', accuracy, precision, recall, f1]], columns=['classifier', 'feature vector', 'accuracy', 'precision', 'recall', 'f1'])
#evaluation_df = evaluation_df.append(metrics)
#print("Xgb, WordLevel TF-IDF: \n accuracy: %s\n precision: %s\n recall: %s\n f1:%s"%(accuracy,precision,recall,f1))

In [115]:
xgb_tfidf = np.array(['Extreme Gradient Boosting', 'TF-IDF', accuracy, precision, recall, f1])

In [120]:
metrics_all = np.array([nb_count, nb_tfidf, lr_count, lr_tfidf, rf_count, rf_tfidf, xgb_count, xgb_tfidf])

In [127]:
metrics_df = pd.DataFrame(metrics_all, columns=['classifier', 'feature vector', 'accuracy', 'precision', 'recall', 'f1'])

In [128]:
metrics_df.to_csv('metrics_scores.csv')

In [126]:
#calculate precision and recall 
#remove sus stocks from list, then do senti anaysis on this 80 20 
#the second accuracy test 

In [129]:
pd.DataFrame(metrics_all, columns=['classifier', 'feature vector', 'accuracy', 'precision', 'recall', 'f1'])

,classifier,feature vector,accuracy,precision,recall,f1
0,Naive Bayes,Count,0.4965034965034965,0.4946524064171123,0.4998860788334472,0.49595959595959593
1,Naive Bayes,TF-IDF,0.4965034965034965,0.4946524064171123,0.4998860788334472,0.49595959595959593
2,Logistic Regression,Count,0.4965034965034965,0.4946524064171123,0.4998860788334472,0.49595959595959593
3,Logistic Regression,TF-IDF,0.4965034965034965,0.4946524064171123,0.4998860788334472,0.49595959595959593
4,Random Forest,Count,0.4965034965034965,0.4946524064171123,0.4998860788334472,0.49595959595959593
5,Random Forest,TF-IDF,0.4965034965034965,0.4946524064171123,0.4998860788334472,0.49595959595959593
6,Extreme Gradient Boosting,Count,0.4965034965034965,0.4946524064171123,0.4998860788334472,0.49595959595959593
7,Extreme Gradient Boosting,TF-IDF,0.4965034965034965,0.4946524064171123,0.4998860788334472,0.49595959595959593
